In [ ]:
#Primero van a necesitar instalarse el siguiente modulo:
#pip install transformers sentencepiece

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModelWithLMHead
from bs4 import BeautifulSoup
import requests

C:\Users\rover\miniconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
URL = 'https://www.theguardian.com/books/2022/nov/05/i-want-to-open-a-window-in-their-souls-haruki-murakami-on-the-power-of-writing-simply'

In [3]:
r = requests.get(URL)

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['p'])
text = [result.text for result in results[1:-1]]
ARTICLE = ' '.join(text)

In [5]:
ARTICLE

'My first novel, Hear the Wind Sing, published in 1979, is fewer than 200 pages long. Yet it took many months and much effort to complete. Part of the reason, of course, was the limited time I had to work on it. I ran a jazz cafe, and I spent my 20s labouring from morning to night to pay off debts. But the real problem was that I hadn’t a clue how to write a novel. To tell the truth, although I had been absorbed in reading all kinds of stuff – my favourites being translations of Russian novels and English-language paperbacks – I had never read modern Japanese novels (of the “serious” variety) in any concerted way. Thus I had no idea what kind of Japanese literature was being read at the time or how I should write fiction in Japanese. For several months, I operated on pure guesswork, adopting what seemed to be a likely style and running with it, but when I read through the result I was far from impressed. “Good grief,” I moaned, “this is hopeless.” What I had written seemed to fulfil th

In [5]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [6]:
sentences = ARTICLE.split('<eos>')

<h3>Sentiment Analysis</h3>

sentiment analysis detection system in which you can select a sentence or parragraph of the article and it will return whether it is positive or negative

In [8]:
sentiment_analysis = pipeline("sentiment-analysis")
res = sentiment_analysis(sentences[3])
res

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'NEGATIVE', 'score': 0.9965852499008179}]

<h3>Summarization</h3>
I have to split the blog in smaller chunks

In [9]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [28]:
max_chunk = 500

In [29]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [33]:
res = summarizer(chunks, max_length=200, min_length=25, do_sample=False)

In [34]:
text_sum = ' '.join([summ['summary_text'] for summ in res])

In [35]:
text_sum

' My first novel, Hear the Wind Sing, published in 1979, is fewer than 200 pages long . It took many months and much effort to complete . But the real problem was that I hadn’t a clue how to write a novel .  Japanese was born and raised in Japan, so the vocabulary and patterns of Japanese – in short, the language’s contents – filled the system that was me to bursting . The language had to be simple, my ideas expressed in an easy-to-understand way, the descriptions stripped of all extraneous fat, the form made compact, and everything arranged to fit a container of limited size . The result was a rough, uncultivated kind of prose, but a distinctive rhythm began to take shape .  I rewrote the “rather boring” novel I had just finished from top to bottom in the new style . Writing in my new style felt more like performing music than composing literature . It was as if the words were coming through my body instead of from my head .  Hear the Wind Sing won a Japanese literary prize for new wr

<h3>Question Generation </h3>

This question generator creates questions based on a paragraph or a sentence of the article and the proper answer, and then it generates a question.

In [25]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

context = text[0]
# tambien se podria escribir cualquier parrafo u oracion
answer = 'Hear the Wind Sing'
get_question(answer, context)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


'<pad> question: What was the name of my first novel?</s>'